## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros


# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=["refined_transactions_fees"],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-12-13 13:51:00 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25 process=51769
2024-12-13 13:51:00 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=51769
2024-12-13 13:51:00 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76 process=51769
2024-12-13 13:51:00 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37 process=51769
2024-12-13 13:51:01 [info     ] prepared 1 input batches.      filename=bydate.py lineno=96 process=51769


## Run the model

This automatically registers the model outputs as duckdb tables.

In [2]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

execute_model_in_memory(
    duckdb_client=duckdb_client,
    model="refined_transactions_fees",
    data_reader=batch,
    limit_input_parquet_files=1
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-12-13 13:51:01 [info     ] Executing model...             filename=testutils.py lineno=220 process=51769
2024-12-13 13:51:01 [info     ] duckdb dataset='ingestion/transactions_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/transactions_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=51769
2024-12-13 13:51:02 [info     ] registered view: 'ingestion_transactions_v1' using 1 parquet paths filename=client.py lineno=53 process=51769
2024-12-13 13:51:02 [info     ] duckdb dataset='ingestion/blocks_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/blocks_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=51769
2024-12-13 13:51:04 [info     ] registered view: 'ingestion_blocks_v1' using 1 parquet paths filename=client.py lineno=53 process=51769
2024-12-13 13:51:04 [info     ] duckdb dataset='ingestion/logs_v1' using 1/22 parquet paths, first path is gs://

┌──────────────────────────────┐
│             name             │
│           varchar            │
├──────────────────────────────┤
│ ingestion_blocks_v1          │
│ ingestion_logs_v1            │
│ ingestion_transactions_v1    │
│ refined_transactions_fees    │
│ refined_transactions_fees_v1 │
└──────────────────────────────┘

## Verify model results

In [3]:
duckdb_client.sql("SELECT * FROM refined_transactions_fees_v1 LIMIT 10")

┌────────────┬─────────┬──────────┬─────────┬──────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬──────────────┬─────────────────┬────────────────────────────────────────────────────────────────────┬───────────┬───────────┬──────────────────┬─────────────────────┬───────────────┬──────────────────────┬──────────────────────────┬──────────────────┬──────────────────────────┬─────────────────────────┬──────────────┬─────────────────┬─────────────┬─────────────────────┬────────────┬─────────┬────────────────────┬─────────────────────────┬──────────────────┬───────────────────┬────────────────────┬───────────────────────┬─────────────────────────────────────┬─────────────┬───────────────┬────────────────────┬───────────────────────────┬─────────────────────┬───────────────────────────┬─────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬─────────────────────

### Check the data output size

In [4]:
duckdb_client.sql("SELECT COUNT(*) FROM refined_transactions_fees_v1")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        33168 │
└──────────────┘

In [5]:
duckdb_client.sql("SELECT COUNT(*) FROM ingestion_transactions_v1")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        33168 │
└──────────────┘

In [6]:
# duckdb_client.sql("SELECT input FROM ingestion_transactions_v1 LIMIT 10")


### You can also convert the results to dataframes to inspect them in more familiar ways

In [7]:
duckdb_client.sql("SELECT * FROM refined_transactions_fees_v1 WHERE transaction_type = 0 ORDER BY tx_fee_native DESC LIMIT 10").pl().head()

dt,chain,chain_id,network,nonce,transaction_index,from_address,to_address,block_number,block_timestamp,hash,gas_price,gas_limit,receipt_gas_used,receipt_l1_gas_used,l1_fee,receipt_l1_gas_price,receipt_l1_blob_base_fee,base_fee_per_gas,max_priority_fee_per_gas,base_legacy_fee_per_gas,l2_fee,l2_priority_fee,l2_base_fee,block_hour,method_id,success,l1_base_fee_scalar,l1_blob_base_fee_scalar,transaction_type,input_byte_length,input_calldata_gas,is_system_transaction,is_attributes_deposited_transaction,l1_gas_used,tx_fee,l2_base_legacy_fee,l1_base_fee,l1_base_scaled_size,l1_blob_fee,l1_blob_scaled_size,tx_fee_native,l1_fee_native,l2_fee_native,l1_base_fee_native,l1_blob_fee_native,l2_base_fee_native,l2_priority_fee_native,l2_base_legacy_fee_native,l2_gas_price_gwei,l2_base_gas_price_gwei,l2_priority_gas_price_gwei,l2_base_legacy_gas_price_gwei,l1_base_gas_price_gwei,l1_blob_base_gas_price_gwei
date,str,i32,str,i64,i64,str,str,i64,u32,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,datetime[μs],str,bool,"decimal[36,7]","decimal[26,7]",i32,f64,f64,bool,bool,i64,i64,i64,"decimal[38,12]","decimal[38,12]","decimal[38,12]","decimal[38,12]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]"
2024-11-18,"""op""",10,"""mainnet""",254942,1,"""0x3088f0191e49f120d567fbd21ae6…","""0xeb4378aacdfb7d02a0c9accfac2e…",128145323,1731889423,"""0xa154c2a27f36f900ba0cc89f30e4…",6009964258,1000000,167161,1600,306301485470,7670139258,2387610658,856,0,6009963402,1004631635331538,0,143089816,2024-11-18 00:00:00,"""0x01000000""",true,0.0836320,1.0142130,0,39.0,492.0,false,false,1600,1004937936817008,1004631492241722,64146908642.505600000000,8.363200000000,242154576828.215400000000,101.421300000000,0.0010049379368170080,0.0000003063014854700,0.0010046316353315380,0.0000000641469086430,0.0000002421545768280,0.0000000001430898160,0.0000000000000000000,0.0010046314922417220,6.0099642580,0.0000008560,0.0000000000,6.0099634020,7.6701392580,2.3876106580
2024-11-18,"""op""",10,"""mainnet""",254890,1,"""0x3088f0191e49f120d567fbd21ae6…","""0xeb4378aacdfb7d02a0c9accfac2e…",128144808,1731888393,"""0x3bf1efa2db347fb47dd054401ef4…",3557316353,1000000,167130,1600,178769154750,8171754126,1088796344,662,0,3557315691,594534282076890,0,110640060,2024-11-18 00:00:00,"""0x01000000""",true,0.0836320,1.0142130,0,39.0,492.0,false,false,1600,594713051231640,594534171436830,68342014106.563200000000,8.363200000000,110427140643.727200000000,101.421300000000,0.0005947130512316400,0.0000001787691547500,0.0005945342820768900,0.0000000683420141070,0.0000001104271406440,0.0000000001106400600,0.0000000000000000000,0.0005945341714368300,3.5573163530,0.0000006620,0.0000000000,3.5573156910,8.1717541260,1.0887963440
2024-11-18,"""op""",10,"""mainnet""",278585,3,"""0xe3e73f1aa65eeb809ff1f41f3c09…","""0x68b3465833fb72a70ecdf485e0e4…",128145787,1731890351,"""0x3c34052b55b8ea87fad65536694c…",3806621788,1000000,150414,2263,496962886328,8132807920,2793617096,1778,0,3806620010,572569209620232,0,267436092,2024-11-18 00:00:00,"""0x04e45aaf""",false,0.0836320,1.0142130,0,228.0,1644.0,false,false,2263,573066172506560,572568942184140,96200553176.111920000000,11.828701000000,400738090100.154301500000,143.447751187500,0.0005730661725065600,0.0000004969628863280,0.0005725692096202320,0.0000000962005531760,0.0000004007380901000,0.0000000002674360920,0.0000000000000000000,0.0005725689421841400,3.8066217880,0.0000017780,0.0000000000,3.8066200100,8.1328079200,2.7936170960
2024-11-18,"""op""",10,"""mainnet""",15578,6,"""0x9c71e8299b145a4190c65ac0d555…","""0x8fc1756b0bc045c06b7bb207b629…",128145605,1731889987,"""0x2f1c10fa888a422c3dafe1d8a16d…",477875164,2499999,1130360,20090,3713537605952,7522450707,2295686953,1299,0,477873865,540170970379040,0,1468337640,2024-11-18 00:00:00,"""0xd9d73414""",true,0.0836320,1.0142130,0,1604.0

### Check Type 0 Legacy Fee Logic

In [8]:
duckdb_client.sql("""SELECT * FROM refined_transactions_fees_v1
                    WHERE transaction_type = 0
                  AND ( --check fee
                        l1_fee + l2_base_fee + l2_priority_fee + l2_base_legacy_fee != tx_fee
                        OR --check prices
                        l2_base_gas_price_gwei + l2_priority_gas_price_gwei + l2_base_legacy_gas_price_gwei != l2_gas_price_gwei
                        )
                  LIMIT 10
                  """).pl().head()

dt,chain,chain_id,network,nonce,transaction_index,from_address,to_address,block_number,block_timestamp,hash,gas_price,gas_limit,receipt_gas_used,receipt_l1_gas_used,l1_fee,receipt_l1_gas_price,receipt_l1_blob_base_fee,base_fee_per_gas,max_priority_fee_per_gas,base_legacy_fee_per_gas,l2_fee,l2_priority_fee,l2_base_fee,block_hour,method_id,success,l1_base_fee_scalar,l1_blob_base_fee_scalar,transaction_type,input_byte_length,input_calldata_gas,is_system_transaction,is_attributes_deposited_transaction,l1_gas_used,tx_fee,l2_base_legacy_fee,l1_base_fee,l1_base_scaled_size,l1_blob_fee,l1_blob_scaled_size,tx_fee_native,l1_fee_native,l2_fee_native,l1_base_fee_native,l1_blob_fee_native,l2_base_fee_native,l2_priority_fee_native,l2_base_legacy_fee_native,l2_gas_price_gwei,l2_base_gas_price_gwei,l2_priority_gas_price_gwei,l2_base_legacy_gas_price_gwei,l1_base_gas_price_gwei,l1_blob_base_gas_price_gwei
date,str,i32,str,i64,i64,str,str,i64,u32,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,datetime[μs],str,bool,"decimal[36,7]","decimal[26,7]",i32,f64,f64,bool,bool,i64,i64,i64,"decimal[38,12]","decimal[38,12]","decimal[38,12]","decimal[38,12]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]"


### Get table schema

In [9]:
duckdb_client.sql("DESCRIBE refined_transactions_fees_v1")


┌───────────────────────────────┬────────────────┬─────────┬─────────┬─────────┬─────────┐
│          column_name          │  column_type   │  null   │   key   │ default │  extra  │
│            varchar            │    varchar     │ varchar │ varchar │ varchar │ varchar │
├───────────────────────────────┼────────────────┼─────────┼─────────┼─────────┼─────────┤
│ dt                            │ DATE           │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                         │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ chain_id                      │ INTEGER        │ YES     │ NULL    │ NULL    │ NULL    │
│ network                       │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ nonce                         │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_index             │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ from_address                  │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │